In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # remove when finished
from implementations import *

In [3]:
data_tr = pd.read_csv('data/train.csv')
idx_tr = data_tr.Id

In [4]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [5]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [6]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi}
    return df_drop_name

In [7]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.152456
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [8]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].value_counts().index[0]#mean(axis=0)
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].value_counts().index[0]#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [9]:
drop_col = drop_columns(df_miss_tr)
drop_col

{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num']}

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [10]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [11]:
feature_tr_grp = group_features_by_jet(data_tr)

In [12]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi'])))

In [49]:
corr0 = feature_tr_grp[0].corr()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,PRI_tau_eta,PRI_lep_pt,PRI_lep_eta,PRI_met,PRI_met_sumet
Id,1.000000,-0.002778,-0.004709,-0.003248,-0.000686,-0.002736,-0.000686,0.001170,-0.000092,0.003763,0.002064,0.007009,-0.000265,0.004961,-0.001881,0.003485
DER_mass_MMC,-0.002778,1.000000,0.082198,0.772553,0.011128,0.519232,0.011128,0.569031,0.066944,0.080372,0.401294,0.011704,0.500168,0.006189,-0.019026,0.198762
DER_mass_transverse_met_lep,-0.004709,0.082198,1.000000,0.082295,-0.020909,-0.303470,-0.020909,0.227126,0.604802,-0.260864,-0.185432,-0.011317,0.560495,-0.013906,0.681983,0.005508
DER_mass_vis,-0.003248,0.772553,0.082295,1.000000,0.015904,0.658827,0.015904,0.658896,0.003557,0.002061,0.501184,0.011192,0.541033,0.004564,-0.025195,0.240522
DER_pt_h,-0.000686,0.011128,-0.020909,0.015904,1.000000,-0.034732,1.000000,0.033572,-0.006264,0.397374,0.032596,0.001699,0.020196,0.012447,0.319307,0.405886
DER_deltar_tau_lep,-0.002736,0.519232,-0.303470,0.658827,-0.034732,1.000000,-0.034732,0.193600,0.045459,0.164731,0.119419,0.017772,0.188038,0.011686,-0.420527,0.049709
DER_pt_tot,-0.000686,0.011128,-0.020909,0.015904,1.000000,-0.034732,1.000000,0.033572,-0.006264,0.397374,0.032596,0.001699,0.020196,0.012447,0.319307,0.405886
DER_sum_pt,0.001170,0.569031,0.227126,0.658896,0.033572,0.193600,0.033572,1.000000,-0.015693,-0.014208,0.800269,-0.005207,0.779744,-0.000066,0.152322,0.383551
DER_pt_ratio_lep_tau,-0.000092,0.066944,0.604802,0.003557,-0.006264,0.045459,-0.006264,-0.015693,1.000000,-0.046202,-0.565337,-0.004896,0.564931,-0.005556,0.162514,-0.092846
DER_met_phi_centrality,0.003763,0.080372,-0.260864,0.002061,0.397374,0.164731,0.397374,-0.014208,-0.046202,1.000000,0.015339,0.009325,-0.038966,0.015513,-0.170603,0.212108


In [12]:
# for df['PRI_jet_num'] == 0
# x_tr0,mean0,std0 = standardize(feature_tr_grp[0].drop(columns = ['Id','Prediction']).values)
# y_tr0 = feature_tr_grp[0].Prediction.replace(['s','b'],[1,-1]).values
# w0 = least_squares(y_tr0,x_tr0)
# idx_tr0 = feature_tr_grp[0].Id
# y_pred_tr0 = predict_labels(w0, x_tr0)


In [13]:
# for df['PRI_jet_num'] == 1
# x_tr1,_,_ = standardize(feature_tr_grp[1].drop(columns = ['Id','Prediction']).values)
# y_tr1 = feature_tr_grp[1].Prediction.replace(['s','b'],[1,-1]).values
# w1 = least_squares(y_tr1,x_tr1)
# idx_tr1 = feature_tr_grp[1].Id
# y_pred_tr1 = predict_labels(w1, x_tr1)


In [14]:
# for df['PRI_jet_num'] == 2,3
# x_tr2,_,_ = standardize(feature_tr_grp[2].drop(columns = ['Id','Prediction']).values)
# y_tr2 = feature_tr_grp[2].Prediction.replace(['s','b'],[1,-1]).values
# w2 = least_squares(y_tr2,x_tr2)
# idx_tr2 = feature_tr_grp[2].Id
# y_pred_tr2 = predict_labels(w2, x_tr2)

In [15]:
# create_csv_submission(idx_te, y_pred_te, 'submission.csv')

In [16]:
# k_fold = 10
# seed = 1
# lambdas = np.logspace(-10, 2, 20) 

# for lambda_ in lambdas:
#     acc_tr0,acc_te0 = cv_loop(y_tr0, x_tr0,k_fold,seed,ridge_regression,lambda_=lambda_)
#     print("lambda = {}: Training accuracy: {} - Test accuracy : {}".format(lambda_, acc_tr0, acc_te0))

In [17]:
# ridge_regression using cv
# w = []
# acc_tr = []
# acc_te = []
# lambda_ = 0.0001
# k_fold = 10
# seed = 1

# for i in range(len(feature_tr_grp)):
#     x_tr,_,_ = standardize(feature_tr_grp[i].drop(columns = ['Id','Prediction']).values)
#     y_tr = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
#     w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,ridge_regression,lambda_=lambda_)
#     w.append(w_tmp)
#     acc_tr.append(acc_tr_tmp)
#     acc_te.append(acc_te_tmp)

In [43]:
k_fold = 10
seed = 10
max_iters = 200
gammas = [0.001,0.01,0.1,0.5,1,1.5,2]

x_tr,_,_ = standardize(feature_tr_grp[2].drop(columns = ['Id','Prediction']).values)
x_tr = np.hstack((np.ones((x_tr.shape[0], 1)), x_tr))
y_tr = feature_tr_grp[2].Prediction.replace(['s','b'],[1,-1]).values

for gamma in gammas:
    loss,w = logistic_regression_GD(y_tr, x_tr, max_iters, gamma)
    y_tr = y_tr.reshape(len(y_tr),1)
    y_tr_pred = predict_labels(w, x_tr)
    acc_tr = compute_accuracy(y_tr_pred,y_tr)
    print("Current gamma={i}, loss={l}, acc = {a}".format(i=gamma, l=loss,a = acc_tr))



# k_indices = build_k_indices(y_tr, k_fold = 4, seed = 10)
# k = 0
# test_idx = k_indices[k]
# train_idx = list(set(np.arange(0,len(y_tr)))-set(k_indices[k]))
# [x_train,y_train,x_test,y_test] = [x_tr[train_idx], y_tr[train_idx], x_tr[test_idx], y_tr[test_idx]]
# w_tmp,acc_tr_tmp,acc_te_tmp = cv_loop(y_tr, x_tr,k_fold,seed,logistic_regression_GD, max_iters=100, gamma=gamma)

Current gamma=0.001, loss=-330935.8837137921, acc = [0.6937954]
Current gamma=0.01, loss=-329995.2038006718, acc = [0.69380919]
Current gamma=0.1, loss=-329851.1063430715, acc = [0.69380919]
Current gamma=0.5, loss=-329833.6974490663, acc = [0.6937954]
Current gamma=1, loss=-329859.1231556807, acc = [0.69380919]
Current gamma=1.5, loss=-329822.3560749686, acc = [0.69380919]
Current gamma=2, loss=-329693.3439553426, acc = [0.69378162]


In [41]:
w.shape

(19, 1)

In [14]:
loss,w = logistic_regression_GD(y_train, x_train, max_iters, gamma)
y_train = y_train.reshape(len(y_train),1)
y_train_pred = predict_labels(w, x_train)
acc_tr = compute_accuracy(y_train_pred,y_train)
print("Current gamma={i}, acc={l}".format(i=gamma, l=acc_tr))

In [15]:
loss

array(-988441.70085384)

In [19]:
y_train = y_train.reshape(len(y_train),1)

In [16]:
y_train_pred = predict_labels(w, x_train)

In [20]:
compute_accuracy(y_train_pred,y_train)

array([0.80200173])

In [15]:
initial_w = np.zeros((x_train.shape[1], 1))
loss_log,w_log = logistic_regression_GD(y_train, x_train, initial_w, max_iters, gamma)